In [ ]:
!wget https://hf.co/danjacobellis/walloc/resolve/main/Stereo_Li_48c_J8_nf8_v1.0.2.pth

In [ ]:
!python3 -m pip install -U git+https://github.com/facebookresearch/demucs#egg=demucs

In [1]:
import torch
import numpy as np
import torchaudio
import tempfile
from cdpam import CDPAM
from demucs.separate import Separator
from IPython.display import Audio
from datasets import load_dataset, Dataset
from walloc import walloc
from spauq.core.metrics import spauq_eval
from fastprogress.fastprogress import progress_bar
class Config: pass

In [2]:
device = "cuda"
checkpoint = torch.load("Stereo_Li_48c_J8_nf8_v1.0.2.pth",map_location="cpu",weights_only=False)
codec_config = checkpoint['config']
codec = walloc.Codec1D(
    channels = codec_config.channels,
    J = codec_config.J,
    Ne = codec_config.Ne,
    Nd = codec_config.Nd,
    latent_dim = codec_config.latent_dim,
    latent_bits = codec_config.latent_bits,
    lightweight_encode = codec_config.lightweight_encode,
    post_filter = codec_config.post_filter
).to(device)
codec.load_state_dict(checkpoint['model_state_dict'])
codec.eval();

/home/dgj335/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [3]:
device = "cuda"
separator = Separator()
cdpam_loss = CDPAM()
MUSDB = load_dataset("danjacobellis/musdb18HQ", split='validation')

/home/dgj335/.local/lib/python3.10/site-packages/cdpam/cdpam.py:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(modfolder,map_location="cpu")['state']


Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

In [4]:
separator = Separator()

In [5]:
def pad(audio, p=2**16):
    B,C,L = audio.shape
    padding_size = (p - (L % p)) % p
    if padding_size > 0:
        audio = torch.nn.functional.pad(audio, (0, padding_size), mode='constant', value=0)
    return audio

In [6]:
max_duration=44100*180
SSDR = []
SRDR = []
PSNR = []
CDPAM_dB = []
res_red = torchaudio.transforms.Resample(44100, 16536).to(device)
res_inc = torchaudio.transforms.Resample(16536, 44100).to(device)
with torch.no_grad():
    for i_sample in progress_bar(range(0, len(MUSDB), 5)):
        SSDR.append([])
        SRDR.append([])
        PSNR.append([])
        CDPAM_dB.append([])
        y = []
        for i_instr in range(5):
            sample = MUSDB[i_sample+i_instr]
            instr = sample['instrument']
            x, fs = torchaudio.load(sample['audio']['bytes'])
            x = x[:,:max_duration]
            L = x.shape[-1]
            x_padded = pad(x.unsqueeze(0), 2**16).to(device)

            X = codec.wavelet_analysis(x_padded,codec.J)
            Y = codec.encoder(X)
            X_hat = codec.decoder(Y)
            x_hat = codec.wavelet_synthesis(X_hat,codec.J)
            x_hat = codec.post(x_hat)
                              
            if i_instr == 0:
                pred = separator.separate_tensor(x_hat[0])
                
            else:
                ℓ = min(x_padded.shape[-1], pred[1][instr].shape[-1])
                SDR = spauq_eval(
                    reference=x_padded[0,:,:ℓ].to("cpu"),
                    estimate=pred[1][instr][:,:ℓ].to("cpu"),
                    fs = fs
                )
                psnr = 20*np.log10(2) - 10*np.log10(torch.nn.functional.mse_loss(
                    x_padded[0,:,:ℓ],
                    pred[1][instr][:,:ℓ]
                ).item())
                cdpam = cdpam_loss.forward(
                    wav_in=x_padded[0,:,:ℓ],
                    wav_out=pred[1][instr][:,:ℓ]
                )
                SSDR[-1].append(SDR['SSR'])
                SRDR[-1].append(SDR['SRR'])
                PSNR[-1].append(psnr)
                CDPAM_dB[-1].append(-np.log10(cdpam.mean().item()))
SSDR = torch.tensor(SSDR)
SRDR = torch.tensor(SRDR)
PSNR = torch.tensor(PSNR)
CDPAM_dB = torch.tensor(CDPAM_dB)

/home/dgj335/.local/lib/python3.10/site-packages/spauq/core/preprocessing.py:325: UserWarning: No forgive_mode specified, defaulting to `none`
  warnings.warn(


In [7]:
metrics = {
    'SSDR': SSDR,
    'SRDR': SRDR,
    'PSNR': PSNR,
    'CDPAM_dB': CDPAM_dB,
}

instruments = ['other', 'drums', 'bass', 'vocals']

data = {}
for metric_name, metric_tensor in metrics.items():
    for i, instr in enumerate(instruments):
        column_name = f"{metric_name}_{instr}"
        data[column_name] = metric_tensor[:, i].tolist()
dataset = Dataset.from_dict(data)

In [8]:
dataset.push_to_hub("danjacobellis/LSDIR_demucs_10x_walloc",split='validation')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/danjacobellis/LSDIR_demucs_2x_walloc/commit/4e46379bc0a52e4b116f37ea9e970c3497014cb3', commit_message='Upload dataset', commit_description='', oid='4e46379bc0a52e4b116f37ea9e970c3497014cb3', pr_url=None, pr_revision=None, pr_num=None)

In [9]:
dataset.to_pandas().mean(axis=0)

SSDR_other         13.995920
SSDR_drums         17.338877
SSDR_bass          15.551265
SSDR_vocals        11.651202
SRDR_other          4.316245
SRDR_drums          7.361906
SRDR_bass           0.542824
SRDR_vocals        -1.698209
PSNR_other         36.728756
PSNR_drums         38.318439
PSNR_bass          41.049592
PSNR_vocals        40.152142
CDPAM_dB_other      4.750398
CDPAM_dB_drums      4.272236
CDPAM_dB_bass       4.970598
CDPAM_dB_vocals     4.660038
dtype: float64

In [10]:
dataset.to_pandas()

,SSDR_other,SSDR_drums,SSDR_bass,SSDR_vocals,SRDR_other,SRDR_drums,SRDR_bass,SRDR_vocals,PSNR_other,PSNR_drums,PSNR_bass,PSNR_vocals,CDPAM_dB_other,CDPAM_dB_drums,CDPAM_dB_bass,CDPAM_dB_vocals
0,12.644656,25.706320,21.457008,7.877296,3.999796,11.743816,10.454283,2.012564,37.825445,39.849909,41.291917,40.092396,3.145706,3.542191,4.449484,4.249564
1,16.844615,12.807218,21.422667,17.075162,6.149382,4.361600,10.544633,8.012726,35.851661,36.898818,43.156792,38.955807,4.879592,3.644716,5.364878,4.384110
2,18.960341,9.836110,11.099253,7.987258,7.228095,2.939537,5.247091,-0.742906,34.738378,38.949034,38.340560,37.916208,5.024901,4.163824,4.570102,4.809789
3,13.248086,19.427580,11.988764,0.021237,4.961112,8.745760,5.057317,-26.321228,35.317647,36.832868,35.042968,40.372030,4.453807,4.356362,4.605040,4.890491
4,17.098453,3.739480,12.949933,6.254681,6.141479,-1.957859,6.501475,0.955852,42.271588,45.878323,43.068100,48.239283,5.267795,5.321034,5.235601,5.134292
5,13.210648,24.699340,27.440637,0.000000,4.650235,12.908952,16.022180,-80.000000,39.684992,41.850050,41.368192,59.231734,4.435305,4.516701,5.096080,5.909911
6,18.134362,23.192102,8.738712,11.322533,6.094488,10.024410,3.653674,4.623855,35.362739,41.114550,43.209003,36.730597,5.052044,4.437821,5.165249,4.391916
7,27.746423,10.667688,21.328081,5.029722,10.450264,4.583307,10.831236,-2.895131,36.474315,42.653648,44.076205,37.587726,5.079650,4.450479,4.243833,4.498865
8,9.282367,9.913414,22.465705,15.117381,1.611914,3.305610,11.650704,6.370111,35.249286,37.863784,41.694512,35.334681,4.684650,4.174923,5.252828,4.434176
9,17.018493,19.282257,6.866209,8.952134,6.601271,9.012079,2.072154,2.848344,35.431917,39.404964,38.842910,36.968080,4.962085,4.137507,4.647900,4.553176
